In [ ]:
# STILL TESTING

from DataAccessCheck import DataChecker
from Scripts.SQLiteFun import create_sqlite_database, execution
from contextlib import closing, redirect_stdout
from IPython.utils.io import Tee

# once finished, try on all HAPI records and export output to file
# input abs path of database file you wish to query from
conn = create_sqlite_database("/home/jovyan/HDRL-Internship-2024/SPASE_Data_20240716.db")

#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE '%/hapi' LIMIT 10"""

# random 5 with multiple prodKeys to try later
"""spase://NASA/NumericalData/AMPTE-CCE/MEPA/PT0.1875S
spase://NASA/NumericalData/DE2/IDM/PT0.25S
spase://NASA/NumericalData/FAST/MAG/Fluxgate/PT7.8125MS
spase://NASA/NumericalData/ISIS1/SFS/Ionogram/PT29S
spase://NASA/NumericalData/MESSENGER/MAG/PT0.05S"""

#i = 0
prodKeys = []
with open("../HAPI_TakeTooLong.txt") as file:
    #prodKeys = file.readlines()
    for line in file:
        #i += 1
        #if ((i>21) and (i<30)):
        prodKeys.append(line.rstrip())
            #continue
            
#prodKeys = execution(HapiStmt, conn)
#print(prodKeys)

# if want output only in file
with open("../DatalinkCheckOutput2.txt", "w") as file:
    with redirect_stdout(file):
        lines = DataChecker(prodKeys, conn)
# if want both file and console
#with closing(Tee("../DatalinkCheckOutput2.txt", "w", channel="stdout")) as outputstream:
 #   lines = DataChecker(prodKeys, conn)
#print("The program is done!")

# export all datasets which take too long to a text file for further investigation
textFile = open("../HAPI_TakeTooLong2.txt", "w")
for line in lines:
    textFile.write(line)
    textFile.write("\n")
textFile.close()

In [ ]:
prodKeys = []
with open("../HAPI_TakeTooLong2.txt") as file:
    #prodKeys = file.readlines()
    for line in file:
        prodKeys.append(line.rstrip())
            
#prodKeys = execution(HapiStmt, conn)
#print(prodKeys)

# if want output only in file
with open("../DatalinkCheckOutput3.txt", "w") as file:
    with redirect_stdout(file):
        lines = DataChecker(prodKeys, conn)
# if want both file and console
#with closing(Tee("../DatalinkCheckOutput3.txt", "w", channel="stdout")) as outputstream:
 #   lines = DataChecker(prodKeys, conn)
print("The program is done!")

# export all datasets which take too long to a text file for further investigation
textFile = open("../HAPI_TakeTooLong3.txt", "w")
for line in lines:
    textFile.write(line)
    textFile.write("\n")
textFile.close()

In [ ]:
# call .py file directly from notebook
#%run ./HAPICheck.py > ../DatalinkCheckOutput.txt

In [ ]:
# clear .txt file or use another file instead
#textFile = open("../HAPI_TakeTooLong2.txt", "w")
#textFile.close()